In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import collections
import cv2
import datetime
import math
import numpy
import os
import pathlib
import re
import skimage.io
import skimage.transform
import sklearn.model_selection
import tensorflow
import time

from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma
)
from IPython.display import Markdown as md
from markdownTable import markdownTable

from files import create_folder, save_fit_history, save_lossgraph, save_figs
from metrics import dice_coef, jaccard_distance
from model import evaluate, unet_model, get_loss_function
from AugmentationSequence import AugmentationSequence

# GPU

In [ ]:
gpus = tensorflow.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(f"GPU: {gpu.name}")
            tensorflow.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
cfg = {
    "channel": 3,
    "batch_size": 4,
    "fold": 5,
    "epochs": 75,
    "image_size": 400,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_dataset": "dataset",
    "path_out": "out",
    "loss_function": "dice"
}
images_folder = os.path.join(cfg["path_dataset"], "original")
masks_folder = os.path.join(cfg["path_dataset"], "mask")

In [ ]:
list_labels = list([])
list_images = list([])
list_images_names = list([])
for file in pathlib.Path(masks_folder).rglob("*"):
    mask = skimage.io.imread(str(file.resolve()))
    mask = skimage.transform.resize(mask, (cfg["image_size"], cfg["image_size"]), anti_aliasing=True)
    mask = numpy.float32(mask > 200)
    list_labels.append(mask)

    image = skimage.io.imread(os.path.join(images_folder, file.name))
    image = skimage.transform.resize(image, (cfg["image_size"], cfg["image_size"]), anti_aliasing=True)
    if cfg["channel"] == 1:
        image = skimage.color.rgb2gray(image)
    image = numpy.float32(image)
    list_images.append(image)

    list_images_names.append(str(file.stem))

print(len(list_labels), len(list_images), len(list_images_names))

In [ ]:
x = numpy.array(list_images).reshape((len(list_images), cfg["image_size"], cfg["image_size"], cfg["channel"]))
y = numpy.array(list_labels).reshape((len(list_labels), cfg["image_size"], cfg["image_size"], 1))

print(x.shape, y.shape)

In [ ]:
kf = sklearn.model_selection.KFold(n_splits=cfg["fold"], shuffle=True, random_state=cfg["random_state"])
l = list([])
for (train_index, test_index) in kf.split(x):
    l = l + test_index.tolist()
print(len(list(set(l))))
print(collections.Counter(sorted(l)))

In [ ]:
models = []
list_evaluate = list([])
current_datetime = datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S')
path = os.path.join(cfg["path_out"], current_datetime)
create_folder(list([path]))
for fold, (train_index, test_index) in enumerate(kf.split(x)):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x_train, y_train, test_size=cfg["val_size"], random_state=cfg["random_state"])

    print(x_train.shape)
    print(x_val.shape)
    print(x_test.shape)
    print(x.shape)

    path_fold = os.path.join(path, str(fold))
    create_folder(list([path_fold]))

    augment = Compose([
        HorizontalFlip(),
        ShiftScaleRotate(rotate_limit=45, border_mode=cv2.BORDER_CONSTANT),
        ElasticTransform(border_mode=cv2.BORDER_CONSTANT),
        RandomBrightness(),
        RandomContrast(),
        RandomGamma()
    ])
    steps_per_epoch = math.ceil(x_train.shape[0] / cfg["batch_size"])
    train_generator = AugmentationSequence(x_train, y_train, cfg["batch_size"], augment)
    reduce_learning_rate = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, verbose=1)
    filename_model = os.path.join(path_fold, "unet.h5")
    checkpointer = tensorflow.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
    strategy = tensorflow.distribute.MirroredStrategy()

    with strategy.scope():
        model = unet_model(cfg)
        adam_opt = tensorflow.keras.optimizers.Adam(learning_rate=cfg["learning_rate"])
        model.compile(optimizer=adam_opt, loss=get_loss_function(cfg["loss_function"]),
                      metrics=[dice_coef, jaccard_distance, tensorflow.keras.metrics.Precision(),
                               tensorflow.keras.metrics.Recall()])

    tensorflow.keras.backend.clear_session()
    start_time = time.time()
    fit = model.fit(train_generator,
              steps_per_epoch=steps_per_epoch,
              epochs=cfg["epochs"],
              validation_data=(x_val, y_val),
               callbacks=[checkpointer, reduce_learning_rate]
              )
    end_time = time.time() - start_time

    save_fit_history(fold, fit, path_fold)
    save_lossgraph(fold, fit, path_fold)
    list_evaluate.append(evaluate(end_time, fold, model, x_train, x_val, x_test, y_train, y_val, y_test))

    models.append(model)

    # model = tensorflow.keras.models.load_model("unet_rgb.h5", custom_objects = {"dice_loss": dice_loss, "dice_coef": dice_coef, "jaccard_distance": jaccard_distance })

    save_figs(cfg, list_images_names, test_index, model, path_fold, x)
tensorflow.keras.backend.clear_session()

In [ ]:
model_cfg = {
    "batch_size": str(cfg["batch_size"]),
    "epochs": str(cfg["epochs"]),
    "learning_rate": str(cfg["learning_rate"]), # sem converter nao imprime o 0.001
    "loss_function": cfg["loss_function"]
}

image_cfg = {
    "channel": str(cfg["channel"]),
    "image_size": str(cfg["image_size"]),
}

test_cfg = {
    "fold": str(cfg["fold"]),
    "test_size": str(cfg["test_size"]),
    "val_size": str(cfg["val_size"]),
    "random_state": str(cfg["random_state"]),
}

other_cfg = {
    "path_dataset": cfg["path_dataset"],
    "path_out": cfg["path_out"]
}

filename_cfg = os.path.join(path, "cfg.md")
with open(filename_cfg, "w") as file:
    file.write("### model\n\n")
    file.write(re.sub(r"```$", "\n```", markdownTable(list([model_cfg])).getMarkdown()))
    file.write("\n\n### image\n\n")
    file.write(re.sub(r"```$", "\n```", markdownTable(list([image_cfg])).getMarkdown()))
    file.write("\n\n### test\n\n")
    file.write(re.sub(r"```$", "\n```", markdownTable(list([test_cfg])).getMarkdown()))
    file.write("\n\n### other\n\n")
    file.write(re.sub(r"```$", "\n```", markdownTable(list([other_cfg])).getMarkdown()))
    file.close()

In [ ]:
def get_mean(key, list_evaluate):
    return str(numpy.mean(list([evaluate[key] for evaluate in list_evaluate])))

def get_std(key, list_evaluate):
    return str(numpy.std(list([evaluate[key] for evaluate in list_evaluate])))

mean_metrics_train = {
    "loss_train": get_mean("loss_train", list_evaluate),
    "dice_train": get_mean("dice_train", list_evaluate),
    "jaccard_train": get_mean("jaccard_train", list_evaluate),
    "precision_train": get_mean("precision_train", list_evaluate),
    "recall_train": get_mean("recall_train", list_evaluate)
}

std_metrics_train = {
    "loss_train": get_std("loss_train", list_evaluate),
    "dice_train": get_std("dice_train", list_evaluate),
    "jaccard_train": get_std("jaccard_train", list_evaluate),
    "precision_train": get_std("precision_train", list_evaluate),
    "recall_train": get_std("recall_train", list_evaluate)
}

mean_metrics_val = {
    "loss_val": get_mean("loss_val", list_evaluate),
    "dice_val": get_mean("dice_val", list_evaluate),
    "jaccard_val": get_mean("jaccard_val", list_evaluate),
    "precision_val": get_mean("precision_val", list_evaluate),
    "recall_val": get_mean("recall_val", list_evaluate)
}

std_metrics_val = {
    "loss_val": get_std("loss_val", list_evaluate),
    "dice_val": get_std("dice_val", list_evaluate),
    "jaccard_val": get_std("jaccard_val", list_evaluate),
    "precision_val": get_std("precision_val", list_evaluate),
    "recall_val": get_std("recall_val", list_evaluate)
}

mean_metrics_test = {
    "loss_test": get_mean("loss_test", list_evaluate),
    "dice_test": get_mean("dice_test", list_evaluate),
    "jaccard_test": get_mean("jaccard_test", list_evaluate),
    "precision_test": get_mean("precision_test", list_evaluate),
    "recall_test": get_mean("recall_test", list_evaluate)
}

std_metrics_test = {
    "loss_test": get_std("loss_test", list_evaluate),
    "dice_test": get_std("dice_test", list_evaluate),
    "jaccard_test": get_std("jaccard_test", list_evaluate),
    "precision_test": get_std("precision_test", list_evaluate),
    "recall_test": get_std("recall_test", list_evaluate)
}

filename_mean_std = os.path.join(path, "mean_std.md")
with open(filename_mean_std, "w") as file:
    file.write("### train\n\n")
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([mean_metrics_train])).getMarkdown()))
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([std_metrics_train])).getMarkdown()))
    file.write("\n\n### val\n\n")
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([mean_metrics_val])).getMarkdown()))
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([std_metrics_val])).getMarkdown()))
    file.write("\n\n### test\n\n")
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([mean_metrics_test])).getMarkdown()))
    file.write(re.sub(r"```$", "\n```\n", markdownTable(list([std_metrics_test])).getMarkdown()))
    file.close()

md(re.sub(r"```$", "\n```", markdownTable(list([mean_metrics_train])).getMarkdown()))

In [ ]:
md(re.sub(r"```$", "\n```", markdownTable(list([std_metrics_train])).getMarkdown()))

In [ ]:
md(re.sub(r"```$", "\n```", markdownTable(list([mean_metrics_val])).getMarkdown()))

In [ ]:
md(re.sub(r"```$", "\n```", markdownTable(list([std_metrics_val])).getMarkdown()))

In [ ]:
md(re.sub(r"```$", "\n```", markdownTable(list([mean_metrics_test])).getMarkdown()))

In [ ]:
md(re.sub(r"```$", "\n```", markdownTable(list([std_metrics_test])).getMarkdown()))

In [ ]:
for evaluate in list_evaluate:
    filename_fold = os.path.join(path, str(evaluate["fold"]), "metrics.md")

    info = {
        "fold": evaluate["fold"],
        "time": evaluate["time"]
    }

    metrics_train = {
        "loss_train": str(evaluate["loss_train"]),
        "dice_train": str(evaluate["dice_train"]),
        "jaccard_train": str(evaluate["jaccard_train"]),
        "precision_train": str(evaluate["precision_train"]),
        "recall_train": str(evaluate["recall_train"]),
    }

    metrics_val = {
        "loss_val": str(evaluate["loss_val"]),
        "dice_val": str(evaluate["dice_val"]),
        "jaccard_val": str(evaluate["jaccard_val"]),
        "precision_val": str(evaluate["precision_val"]),
        "recall_val": str(evaluate["recall_val"]),
    }

    metrics_test = {
        "loss_test": str(evaluate["loss_test"]),
        "dice_test": str(evaluate["dice_test"]),
        "jaccard_test": str(evaluate["jaccard_test"]),
        "precision_test": str(evaluate["precision_test"]),
        "recall_test": str(evaluate["recall_test"]),
    }

    with open(filename_fold, "w") as file:
        file.write("### info\n\n")
        file.write(re.sub(r"```$", "\n```", markdownTable(list([info])).getMarkdown()))
        file.write("\n\n### train\n\n")
        file.write(re.sub(r"```$", "\n```", markdownTable(list([metrics_train])).getMarkdown()))
        file.write("\n\n### val\n\n")
        file.write(re.sub(r"```$", "\n```", markdownTable(list([metrics_val])).getMarkdown()))
        file.write("\n\n### test\n\n")
        file.write(re.sub(r"```$", "\n```", markdownTable(list([metrics_test])).getMarkdown()))
        file.close()